# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

## Some global difinitions

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

file_path_list=[]

for root, dirs, files in os.walk(".", topdown=False):
    for name in files:
        path = os.path.join(root, name)
        if path.endswith('.csv'):
            file_path_list.append(path)
file_path_list

/home/workspace


['./event_data/.ipynb_checkpoints/2018-11-03-events-checkpoint.csv',
 './event_data/.ipynb_checkpoints/2018-11-02-events-checkpoint.csv',
 './event_data/.ipynb_checkpoints/2018-11-01-events-checkpoint.csv',
 './event_data/2018-11-27-events.csv',
 './event_data/2018-11-04-events.csv',
 './event_data/2018-11-07-events.csv',
 './event_data/2018-11-09-events.csv',
 './event_data/2018-11-19-events.csv',
 './event_data/2018-11-05-events.csv',
 './event_data/2018-11-22-events.csv',
 './event_data/2018-11-16-events.csv',
 './event_data/2018-11-26-events.csv',
 './event_data/2018-11-24-events.csv',
 './event_data/2018-11-29-events.csv',
 './event_data/2018-11-15-events.csv',
 './event_data/2018-11-20-events.csv',
 './event_data/2018-11-06-events.csv',
 './event_data/2018-11-18-events.csv',
 './event_data/2018-11-21-events.csv',
 './event_data/2018-11-10-events.csv',
 './event_data/2018-11-23-events.csv',
 './event_data/2018-11-02-events.csv',
 './event_data/2018-11-28-events.csv',
 './event_dat

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8353


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

7087


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# DROP TABLES DEFINITIONS

session_history_table_drop = "DROP TABLE IF EXISTS session_history;"
users_info_table_drop = "DROP TABLE IF EXISTS users_info;"
users_listening_history_table_drop = "DROP TABLE IF EXISTS users_listening_history;"

drop_table_queries = [session_history_table_drop, users_info_table_drop, users_listening_history_table_drop]

In [9]:
## Dropping Tables if they exist
for query in drop_table_queries:
    try:
        session.execute(query)
    except Exception as e:
        print(e)

## Creating the tables

#### For the first table 'session_history', we need to model the data according to the query we should perform, hence we will choose our primary key to be a composite key (sessionId, itemInSession)

In [10]:
session_history_table_create = ("""CREATE TABLE IF NOT EXISTS session_history(
                            sessionId int,
                            itemInSession int,
                            artist text, 
                            song_title text, 
                            song_length double, 
                            PRIMARY KEY(sessionId, itemInSession));
""")

#### As for the second table 'users_info', the query requres us to use 'itemInSession' as a clustering column since we need the returned data to be sorted by this column. However, our primary key is still the composite key (sessionId, itemInSession)

In [11]:
users_info_table_create = ("""CREATE TABLE IF NOT EXISTS users_info(
                        userId int, 
                        sessionId int,
                        itemInSession int,
                        artist text, 
                        song_title text,
                        first_name text,
                        last_name text,
                        PRIMARY KEY((userId, sessionId), itemInSession));
""")

#### For the last table 'users_listening_history' about the songs users have listend to, we are concerned about the song_title as the query requires us to return it, but 'song_title' is not unique on its own unless we use it as a composite key with the 'userId', so our primary key becomes (song_title, userId).

In [12]:
users_listening_history_table_create = ("""CREATE TABLE IF NOT EXISTS users_listening_history(
                        userId int,
                        song_title text,
                        first_name text,
                        last_name text,
                        PRIMARY KEY(song_title, userId));
""")

In [13]:
create_table_queries = [session_history_table_create, users_info_table_create, users_listening_history_table_create]

In [14]:
## Creating Tables
for query in create_table_queries:
    try:
        session.execute(query)
    except Exception as e:
        print(e)

## Inserting the data into the tables

In [15]:
# open song file
df = pd.read_csv('event_datafile_new.csv', encoding='utf8')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Mynt,Celeste,F,2,Williams,166.94812,free,"Klamath Falls, OR",52,Playa Haters,53
1,Taylor Swift,Celeste,F,3,Williams,230.47791,free,"Klamath Falls, OR",52,You Belong With Me,53
2,Amy Winehouse,Celeste,F,4,Williams,229.85098,free,"Klamath Falls, OR",52,Valerie,53
3,Jimmy Eat World,Celeste,F,5,Williams,285.83138,free,"Klamath Falls, OR",52,Dizzy,53
4,Maldita Nerea,Anabelle,F,0,Simpson,241.16200,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",158,Supelicula,69


In [16]:
# INSERT RECORDS QUERIES

session_history_table_insert = ("""INSERT INTO session_history (sessionId, itemInSession, artist, song_title, song_length)
                            VALUES (%s, %s, %s, %s, %s);
""")

users_info_table_insert = ("""INSERT INTO users_info (userId, sessionId, itemInSession, artist, song_title, first_name, last_name)
                        VALUES (%s, %s, %s, %s, %s, %s, %s);
""")

users_listening_history_table_insert = ("""INSERT INTO users_listening_history (userId, song_title, first_name, last_name)
                        VALUES (%s, %s, %s, %s);
""")

In [ ]:
for i in range(len(df)):
    # insert table1 record
    table1 = df[['sessionId', 'itemInSession', 'artist', 'song', 'length']].values[i].tolist()
    session.execute(session_history_table_insert, table1)

    # insert table2 record
    table2 = df[['userId', 'sessionId', 'itemInSession', 'artist', 'song', 'firstName', 'lastName']].values[i].tolist()
    session.execute(users_info_table_insert, table2)

    # insert table3 record
    table3 = df[['userId', 'song', 'firstName', 'lastName']].values[i].tolist()
    session.execute(users_listening_history_table_insert, table3)

## Implementing the queries

### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song_title , song_length FROM session_history WHERE sessionId = 338 AND itemInSession = 4 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query2 = "SELECT artist, song_title, first_name, last_name, itemInSession FROM users_info WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query3 = "SELECT first_name, last_name FROM users_listening_history WHERE song_title= 'All Hands Against His Own' ALLOW FILTERING"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)
                    

### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [ ]:
for query in drop_table_queries:
    try:
        session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()

### Deleting the created CSV file to be able to run the notebook again

In [ ]:
os.remove('event_datafile_new.csv')